# Import libraries and original COMPAS data

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tempfile
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

# Download the database from GitHub
github_url = 'https://github.com/propublica/compas-analysis/raw/master/compas.db'

# Create a temporary file to store the database
with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as tmp_file:
    tmp_db_path = tmp_file.name
    urllib.request.urlretrieve(github_url, tmp_db_path)

# Connect to the SQLite database
conn = sqlite3.connect(tmp_db_path)

# Read all tables from the database
arrest = pd.read_sql_query("SELECT * FROM casearrest", conn)
compas = pd.read_sql_query("SELECT * FROM compas", conn)
charge = pd.read_sql_query("SELECT * FROM charge", conn)
people = pd.read_sql_query("SELECT * FROM people", conn)

conn.close()


# Check the compas table

## Multiple records for the same person

In [2]:
compass_r = compas[compas.type_of_assessment == 'Risk of Recidivism']
compass_r.groupby('person_id').size().value_counts()
compass_r.person_id.nunique()

11757

In [3]:
compass_r.groupby(['person_id', 'compas_case_id']).size().value_counts()  

1    11688
2      380
3       26
Name: count, dtype: int64

In [4]:
compass_r.groupby(['person_id', 'compas_case_id']).size().sort_values(ascending = False)

person_id  compas_case_id
4640       71216             3
3071       52814             3
5043       52011             3
4660       52139             3
10131      53568             3
                            ..
3983       71399             1
3984       52708             1
3985       3568              1
3986       69913             1
11757      62760             1
Length: 12094, dtype: int64

In [5]:
compass_r.groupby('person_id').size().sort_values(ascending = False)

person_id
5043     4
11289    4
6935     4
5479     4
2047     4
        ..
4054     1
4055     1
4056     1
4057     1
11757    1
Length: 11757, dtype: int64

## Keep only the first records

In [6]:
compass_r1 = \
compass_r.groupby('person_id'
        ).apply(lambda x: x.sort_values('screening_date').iloc[0]
        ).drop(columns = ['person_id']
        ).reset_index()

compass_r1['screening_date'] = pd.to_datetime(compass_r1.screening_date)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/1423964539.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(lambda x: x.sort_values('screening_date').iloc[0]


In [7]:
compass_r.groupby('person_id').size().value_counts().sum()

11757

# Check the arrest table

In [8]:
arrest['arrest_date'] = pd.to_datetime(arrest.arrest_date)
arrest.head()

,id,name,case_number,arrest_id,arrest_date,charge_degree,days_since_compas_arrest,person_id
0,1,aajah herrington,13005832CF10A,3026755,2013-04-23,(F3),1,533
1,2,aajah herrington,13005832CF10A,3026756,2013-04-23,(M2),1,533
2,3,aajah herrington,14007863MM10A,3144392,2014-05-13,(M1),-384,533
3,4,aaliyah lovo,13015312CF10A,3084831,2013-11-03,(F2),1,900
4,5,aaliyah lovo,13015312CF10A,3084832,2013-11-03,(M1),1,900


# Join compas and charge table together

In [9]:
charge['offense_date'] = pd.to_datetime(charge.offense_date)
charge.columns

Index(['id', 'case_type', 'case_number', 'filing_date', 'offense_date',
       'charge_number', 'charge_degree', 'charge', 'date_charge_filed',
       'filing_type', 'filing_agency', 'name', 'days_since_compas', 'statute',
       'person_id'],
      dtype='object')

In [10]:
charge.head()

,id,case_type,case_number,filing_date,offense_date,charge_number,charge_degree,charge,date_charge_filed,filing_type,filing_agency,name,days_since_compas,statute,person_id
0,1,,13005832CF10A,1800-01-01 00:00:00.000000,2013-04-23,1,(F3),Driving While License Revoked,2013-04-23 00:00:00,No Information/ Transfer to Misdemeanor,Fort Lauderdale PD,aajah herrington,1,322.34(5),533
1,2,,13005832CF10A,1800-01-01 00:00:00.000000,2013-04-23,2,(M2),Petit Theft,2013-04-23 00:00:00,Transfer to County Court,Fort Lauderdale PD,aajah herrington,1,812.014(3)(a),533
2,3,,14042370TC10A,1800-01-01 00:00:00.000000,2014-11-11,1,(M2),Driving License Suspended,2014-11-11 00:00:00,None,Fort Lauderdale PD,aajah herrington,-566,322.34(2),533
3,4,,14007863MM10A,1800-01-01 00:00:00.000000,2014-05-13,1,(M1),Extradition/Defendants,2014-05-13 00:00:00,Fugitive,Fort Lauderdale PD,aajah herrington,-384,941.02,533
4,5,,11065568TI30A,1800-01-01 00:00:00.000000,2011-07-03,1,(0),Violation Of Move Over Law,2011-07-03 00:00:00,None,Broward Sheriff Office,aajah herrington,661,316.126(1)(B)1,533


In [11]:
for i in ['statute', 'charge_degree', 'charge']:

    charge['F_%s' % i], charge['M_%s' % i], charge['other_%s' % i] = charge[i], charge[i], charge[i]

    charge.loc[~charge.charge_degree.str.startswith('(F'), 'F_%s' % i] = None

    charge.loc[~charge.charge_degree.str.match(r'^\(M\d'), 'M_%s' % i] = None

    charge.loc[charge.charge_degree.str.match(r'^\([FM]\d'), 'other_%s' % i] = None

charge.head()


,id,case_type,case_number,filing_date,offense_date,charge_number,charge_degree,charge,date_charge_filed,filing_type,...,person_id,F_statute,M_statute,other_statute,F_charge_degree,M_charge_degree,other_charge_degree,F_charge,M_charge,other_charge
0,1,,13005832CF10A,1800-01-01 00:00:00.000000,2013-04-23,1,(F3),Driving While License Revoked,2013-04-23 00:00:00,No Information/ Transfer to Misdemeanor,...,533,322.34(5),None,None,(F3),None,None,Driving While License Revoked,None,None
1,2,,13005832CF10A,1800-01-01 00:00:00.000000,2013-04-23,2,(M2),Petit Theft,2013-04-23 00:00:00,Transfer to County Court,...,533,None,812.014(3)(a),None,None,(M2),None,None,Petit Theft,None
2,3,,14042370TC10A,1800-01-01 00:00:00.000000,2014-11-11,1,(M2),Driving License Suspended,2014-11-11 00:00:00,None,...,533,None,322.34(2),None,None,(M2),None,None,Driving License Suspended,None
3,4,,14007863MM10A,1800-01-01 00:00:00.000000,2014-05-13,1,(M1),Extradition/Defendants,2014-05-13 00:00:00,Fugitive,...,533,None,941.02,None,None,(M1),None,None,Extradition/Defendants,None
4,5,,11065568TI30A,1800-01-01 00:00:00.000000,2011-07-03,1,(0),Violation Of Move Over Law,2011-07-03 00:00:00,None,...,533,None,None,316.126(1)(B)1,None,None,(0),None,None,Violation Of Move Over Law


## Combine the charge table by case

In [12]:
compass_r1.columns


Index(['person_id', 'id', 'first', 'last', 'compas_person_id',
       'compas_case_id', 'compas_assessment_id', 'agency_text', 'scale_set',
       'assessment_reason', 'legal_status', 'custody_status', 'marital_status',
       'screening_date', 'rec_supervision_level', 'rec_supervision_level_text',
       'score_text', 'scale_id', 'type_of_assessment', 'raw_score',
       'decile_score'],
      dtype='object')

## Create statute variable with degrees

### Explore how to manage the variables

In [13]:
s1 = \
"""
784	Assault/Battery
812	Theft/Robery
810	Burglary and threspass
322	Driver license
831	Forgery and counterfeiting
316	Motor vehicles
893	Drug abuse
817	Fradulent
843	Obstructing justice
"""

chapter_crime_map = {i.split('\t')[0] : i.split('\t')[1] for i in s1.split('\n') if len(i) > 0}

In [14]:
charge_statute_chapter = charge.statute.str.split('.').str[0].map(chapter_crime_map)

In [15]:
pd.Series(' '.join(charge.charge.astype(str)[charge.statute.fillna('').str.contains('817')
                                            ].tolist()).split()).value_counts().head(10)

Use           1230
of            1177
Personal      1028
Info          1027
ID            1023
Crim          1023
Card           383
Credit         358
Attempt        191
Fraudulent     182
Name: count, dtype: int64

In [16]:
charge.charge.astype(str)[charge.statute.fillna('').str.contains('893')]

13                    Possession of Cocaine
14                 Possession Of Clonazepam
25                    Possession of Cocaine
26        Possess Cannabis/20 Grams Or Less
27               Possession Of Mdma W/I/D/S
                        ...                
147993                Possession of Cocaine
147996         Possess Cannabis W/i Deliver
147999    Possession of Cannabis (inactive)
148023             Possession Of Alprazolam
148030    Possess Cannabis/20 Grams Or Less
Name: charge, Length: 20272, dtype: object

In [17]:
chapter_crime_map

{'784': 'Assault/Battery',
 '812': 'Theft/Robery',
 '810': 'Burglary and threspass',
 '322': 'Driver license',
 '831': 'Forgery and counterfeiting',
 '316': 'Motor vehicles',
 '893': 'Drug abuse',
 '817': 'Fradulent',
 '843': 'Obstructing justice'}

In [18]:
(charge_statute_chapter + charge.charge_degree
).value_counts(normalize = True
).round(2).cumsum()

Motor vehicles(0)                 0.33
Driver license(M2)                0.43
Drug abuse(M1)                    0.50
Drug abuse(F3)                    0.57
Theft/Robery(F3)                  0.62
Assault/Battery(M1)               0.67
Driver license(0)                 0.71
Obstructing justice(M1)           0.75
Assault/Battery(F3)               0.77
Motor vehicles(M1)                0.79
Burglary and threspass(F3)        0.81
Drug abuse(F2)                    0.83
Theft/Robery(M2)                  0.85
Driver license(F3)                0.87
Burglary and threspass(F2)        0.89
Theft/Robery(F2)                  0.90
Theft/Robery(M1)                  0.91
Fradulent(F3)                     0.92
Motor vehicles(M2)                0.93
Assault/Battery(F2)               0.94
Burglary and threspass(M1)        0.95
Forgery and counterfeiting(F3)    0.96
Obstructing justice(F3)           0.97
Burglary and threspass(M2)        0.98
Drug abuse(F1)                    0.99
Driver license(M1)       

In [19]:
charge.statute.str.split('.').str[0].map(chapter_crime_map).value_counts()

statute
Motor vehicles                45858
Driver license                20283
Drug abuse                    20269
Theft/Robery                  12189
Assault/Battery               10258
Burglary and threspass         6860
Obstructing justice            5510
Fradulent                      1695
Forgery and counterfeiting      910
Name: count, dtype: int64

In [20]:
pd.concat(
    [charge.statute.str.split('.'
     ).str[0].value_counts(normalize = True
     ), charge.statute.str.split('.'
     ).str[0].value_counts()], axis = 1
).head(50)


,proportion,count
statute,,
316,0.310874,45858
322,0.137500,20283
893,0.137405,20269
812,0.082630,12189
784,0.069540,10258
320,0.050450,7442
810,0.046504,6860
843,0.037353,5510
817,0.011491,1695


### Create the Statute and degree variables

In [21]:
statute_degree = charge.statute.str.split('.').str[0] + charge.charge_degree

In [22]:
statute_degree.value_counts()

316(0)             41017
322(M2)            11915
893(M1)             8764
893(F3)             8551
812(F3)             6229
                   ...  
OPBD9009(MO3)          1
OFLD28152G(MO3)        1
OFLD28152A(MO3)        1
OCSD44A(MO3)           1
OHWD9208B(MO3)         1
Name: count, Length: 627, dtype: int64

In [23]:
statute_degree

0         322(F3)
1         812(M2)
2         322(M2)
3         941(M1)
4          316(0)
           ...   
148081    322(M2)
148082    320(M2)
148083    322(M2)
148084    320(M2)
148085     316(0)
Length: 148086, dtype: object

### Create the map between statute chapter and actual charges

In [24]:
statute_keywords = \
charge.groupby(charge.statute.str.split('.').str[0]
        ).apply(
    lambda x: pd.Series({
        'count':    x.shape[0],
        'keywords': ' '.join(pd.Series(' '.join(x.charge.astype(str).tolist()).split()).value_counts().index[:10].tolist())
    }
             )
        ).sort_values('count', ascending = False)

In [25]:
statute_keywords.shape
statute_keywords.head(30)

,count,keywords
statute,,
316,45858,Fail Speed To Safety Wear Belt/Operator Pers/I...
322,20283,License Suspended Driving W/O Valid Operating ...
893,20269,Possess Possession Cocaine of Grams Or Cannabi...
812,12189,Theft Grand in Degree the 3rd Petit Robbery Pr...
784,10258,Battery Aggravated Assault Felony Law Officer ...
320,7442,Expired Fail Tag/Infraction Unlaw Attach LicTa...
810,6860,Burglary Dwelling Unoccup Trespass Unoccupied ...
843,5510,Resist/Obstruct Violence W/O Officer Resist w/...
817,1695,Use of Personal Info ID Crim Card Credit Attem...


## Combine the charge table

In [26]:
charge['statute_degree'] = statute_degree

In [27]:
charge.statute_degree.head()

0    322(F3)
1    812(M2)
2    322(M2)
3    941(M1)
4     316(0)
Name: statute_degree, dtype: object

In [28]:
combine_col_list = ['offense_date','charge_degree', 'charge', 'filing_type', 'filing_agency',
                    'statute', 'F_statute', 'M_statute', 'other_statute',
                    'F_charge_degree', 'M_charge_degree', 'other_charge_degree', 
                    'F_charge', 'M_charge', 'other_charge', 'statute_degree']

charge_combine = \
charge.groupby(['person_id', 'name', 'case_number']
    ).agg( {i: lambda x: [i for i in list(x) if i is not None] for i in combine_col_list}
    )

charge_combine = charge_combine.reset_index()

charge_combine.head(20)

,person_id,name,case_number,offense_date,charge_degree,charge,filing_type,filing_agency,statute,F_statute,M_statute,other_statute,F_charge_degree,M_charge_degree,other_charge_degree,F_charge,M_charge,other_charge,statute_degree
0,1,miguel hernandez,09083797TI30A,[2009-08-11 00:00:00],[(0)],[Unlawful Speed (Requires Speeds)],[],[Florida Highway Patrol-Ft Lauderdale],[316.187(3)],[],[],[316.187(3)],[],[],[(0)],[],[],[Unlawful Speed (Requires Speeds)],[316(0)]
1,1,miguel hernandez,09098832TI20A,[2009-10-24 00:00:00],[(0)],[Speed/65 Interstate],[],[Florida Highway Patrol-Ft Lauderdale],[316.187(2)(A)],[],[],[316.187(2)(A)],[],[],[(0)],[],[],[Speed/65 Interstate],[316(0)]
2,1,miguel hernandez,13009443TI30A,"[2013-01-14 00:00:00, 2013-01-14 00:00:00]","[(0), (0)]","[Disobey/Ran Stop Sign, Expired Tag/Infraction]",[],"[Broward Sheriff Office, Broward Sheriff Office]","[316.123(2)(A), 320.07(3)(A)]",[],[],"[316.123(2)(A), 320.07(3)(A)]",[],[],"[(0), (0)]",[],[],"[Disobey/Ran Stop Sign, Expired Tag/Infraction]","[316(0), 320(0)]"
3,1,miguel hernandez,13011352CF10A,"[2013-08-13 00:00:00, 2013-08-13 00:00:00]","[(F3), (M1)]","[Aggravated Assault w/Firearm, Battery]","[SAO Information, SAO Information]","[Broward Sheriff Office, Broward Sheriff Office]","[784.021, 784.03]",[784.021],[784.03],[],[(F3)],[(M1)],[],[Aggravated Assault w/Firearm],[Battery],[],"[784(F3), 784(M1)]"
4,1,miguel hernandez,13068564TI20A,[2013-11-08 00:00:00],[(0)],[Unlawful Speed (Requires Speeds)],[],[Florida Highway Patrol-Ft Lauderdale],[316.187(3)],[],[],[316.187(3)],[],[],[(0)],[],[],[Unlawful Speed (Requires Speeds)],[316(0)]
5,1,miguel hernandez,14014411TI20A,[2013-11-20 00:00:00],[(0)],[Red Light Camera Violation],[],[Sunrise PD],[316.075(1)(C)(1)],[],[],[316.075(1)(C)(1)],[],[],[(0)],[],[],[Red Light Camera Violation],[316(0)]
6,3,kevon dixon,13001275CF10A,[2013-01-26 00:00:00],[(F3)],[Felony Battery w/Prior Convict],[SAO Information],[Broward Sheriff Office/Pompano Beach],[784.03(2)],[784.03(2)],[],[],[(F3)],[],[],[Felony Battery w/Prior Convict],[],[],[784(F3)]
7,3,kevon dixon,13009779CF10A,"[2013-07-05 00:00:00, 2013-07-05 00:00:00, 201...","[(F2), (F3), (F3)]","[Sexual Battery / Vict 12 Yrs +, Felony Batter...","[SAO Information, SAO Information, SAO Informa...","[Broward Sheriff Office/Pompano Beach, Broward...","[794.011(5), 784.041(2)(a), 787.02(1)(a)]","[794.011(5), 784.041(2)(a), 787.02(1)(a)]",[],[],"[(F2), (F3), (F3)]",[],[],"[Sexual Battery / Vict 12 Yrs +, Felony Batter...",[],[],"[794(F2), 784(F3), 787(F3)]"
8,4,ed philo,09024308TI20A,"[2009-02-27 00:00:00, 2009-02-27 00:00:00]","[(0), (0)]","[Speed Posted Municip/State Rd, Learner's Driv...",[],"[Broward Sheriff Office, Broward Sheriff Office]","[316.189, 322.1615(2)(B)]",[],[],"[316.189, 322.1615(2)(B)]",[],[],"[(0), (0)]",[],[],"[Speed Posted Municip/State Rd, Learner's Driv...","[316(0), 322(0)]"
9,4,ed philo,09057909TI20A,[2009-06-18 00:00:00],[(0)],[Fnt St Pass/Ovr 18/Stbelt],[],[Broward Sheriff Office],[316.614(5)],[],[],[316.614(5)],[],[],[(0)],[],[],[Fnt St Pass/Ovr 18/Stbelt],[316(0)]


In [29]:
charge_combine.statute_degree.head(10)

0                       [316(0)]
1                       [316(0)]
2               [316(0), 320(0)]
3             [784(F3), 784(M1)]
4                       [316(0)]
5                       [316(0)]
6                      [784(F3)]
7    [794(F2), 784(F3), 787(F3)]
8               [316(0), 322(0)]
9                       [316(0)]
Name: statute_degree, dtype: object

## Combine the arrest table

In [30]:
arrest_combine = arrest.groupby(['person_id', 'name','arrest_date', 'case_number']
                    ).agg( {'days_since_compas_arrest': lambda x: list(set(x))[0]}
                    ).reset_index()
                    

## Combine the arrest and charge table

In [31]:
charge_combine.columns

arrest_combine.columns

Index(['person_id', 'name', 'arrest_date', 'case_number',
       'days_since_compas_arrest'],
      dtype='object')

In [32]:
arrest_charge_combine = arrest_combine.merge(charge_combine.drop(columns = ['person_id', 'name']), on = 'case_number')

arrest_charge_combine


,person_id,name,arrest_date,case_number,days_since_compas_arrest,offense_date,charge_degree,charge,filing_type,filing_agency,...,F_statute,M_statute,other_statute,F_charge_degree,M_charge_degree,other_charge_degree,F_charge,M_charge,other_charge,statute_degree
0,1,miguel hernandez,2013-08-13,13011352CF10A,1,"[2013-08-13 00:00:00, 2013-08-13 00:00:00]","[(F3), (M1)]","[Aggravated Assault w/Firearm, Battery]","[SAO Information, SAO Information]","[Broward Sheriff Office, Broward Sheriff Office]",...,[784.021],[784.03],[],[(F3)],[(M1)],[],[Aggravated Assault w/Firearm],[Battery],[],"[784(F3), 784(M1)]"
1,1,miguel hernandez,2014-07-07,13011352CF10A,-327,"[2013-08-13 00:00:00, 2013-08-13 00:00:00]","[(F3), (M1)]","[Aggravated Assault w/Firearm, Battery]","[SAO Information, SAO Information]","[Broward Sheriff Office, Broward Sheriff Office]",...,[784.021],[784.03],[],[(F3)],[(M1)],[],[Aggravated Assault w/Firearm],[Battery],[],"[784(F3), 784(M1)]"
2,3,kevon dixon,2013-01-26,13001275CF10A,1,[2013-01-26 00:00:00],[(F3)],[Felony Battery w/Prior Convict],[SAO Information],[Broward Sheriff Office/Pompano Beach],...,[784.03(2)],[],[],[(F3)],[],[],[Felony Battery w/Prior Convict],[],[],[784(F3)]
3,3,kevon dixon,2013-07-09,13009779CF10A,-163,"[2013-07-05 00:00:00, 2013-07-05 00:00:00, 201...","[(F2), (F3), (F3)]","[Sexual Battery / Vict 12 Yrs +, Felony Batter...","[SAO Information, SAO Information, SAO Informa...","[Broward Sheriff Office/Pompano Beach, Broward...",...,"[794.011(5), 784.041(2)(a), 787.02(1)(a)]",[],[],"[(F2), (F3), (F3)]",[],[],"[Sexual Battery / Vict 12 Yrs +, Felony Batter...",[],[],"[794(F2), 784(F3), 787(F3)]"
4,3,kevon dixon,2013-07-10,13001275CF10A,-164,[2013-01-26 00:00:00],[(F3)],[Felony Battery w/Prior Convict],[SAO Information],[Broward Sheriff Office/Pompano Beach],...,[784.03(2)],[],[],[(F3)],[],[],[Felony Battery w/Prior Convict],[],[],[784(F3)]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71326,11755,dieuseul pierre-gilles,2014-10-07,14014708MM10A,1,[2014-10-07 00:00:00],[(M1)],[Battery],[SAO Information],[Broward Sheriff Office/Deerfield Beach],...,[],[784.03(1)],[],[],[(M1)],[],[],[Battery],[],[784(M1)]
71327,11756,scott lomagistro,2012-02-19,12003460MM10A,653,[2012-02-19 00:00:00],[(M1)],[Giving False Crime Report],[SAO Information],[Broward Sheriff Office/Pompano Beach],...,[],[817.49],[],[],[(M1)],[],[],[Giving False Crime Report],[],[817(M1)]
71328,11756,scott lomagistro,2012-10-06,12003460MM10A,423,[2012-02-19 00:00:00],[(M1)],[Giving False Crime Report],[SAO Information],[Broward Sheriff Office/Pompano Beach],...,[],[817.49],[],[],[(M1)],[],[],[Giving False Crime Report],[],[817(M1)]
71329,11756,scott lomagistro,2013-05-21,12018116MM10A,196,"[2012-06-04 00:00:00, 2012-06-04 00:00:00, 201...","[(M1), (M1), (0)]","[DUI Property Damage/Injury, DUI Property Dama...","[SAO Information- Not In Custody, SAO Informat...","[Florida Highway Patrol-Ft Lauderdale, Florida...",...,[],"[316.193(3)(A)(B)( C)1, 316.193(3)(A)(B)( C)1]",[316.1925],[],"[(M1), (M1)]",[(0)],[],"[DUI Property Damage/Injury, DUI Property Dama...",[Careless Driving],"[316(M1), 316(M1), 316(0)]"


### Further combine multiple charge in one arrest together  

In [33]:
arrest_charge_combine.columns

Index(['person_id', 'name', 'arrest_date', 'case_number',
       'days_since_compas_arrest', 'offense_date', 'charge_degree', 'charge',
       'filing_type', 'filing_agency', 'statute', 'F_statute', 'M_statute',
       'other_statute', 'F_charge_degree', 'M_charge_degree',
       'other_charge_degree', 'F_charge', 'M_charge', 'other_charge',
       'statute_degree'],
      dtype='object')

In [34]:
d1 = {x: lambda x: [j for i in list(x) for j in i] for x in ['case_number', 'offense_date', 'charge_degree',
                                                       'charge', 'filing_type', 'filing_agency', 'statute',
                                                       'F_statute', 'M_statute', 'other_statute',
                                                       'F_charge_degree', 'M_charge_degree', 'other_charge_degree', 
                                                       'F_charge', 'M_charge', 'other_charge', 'statute_degree']}

d2 = {x: lambda x: list(x) for x in ['case_number']}

arrest_charge_combine2 = \
arrest_charge_combine.groupby(['person_id', 'name', 'arrest_date']
                ).agg( {**d1, **d2}
                )

In [35]:
arrest_charge_combine2.columns

Index(['case_number', 'offense_date', 'charge_degree', 'charge', 'filing_type',
       'filing_agency', 'statute', 'F_statute', 'M_statute', 'other_statute',
       'F_charge_degree', 'M_charge_degree', 'other_charge_degree', 'F_charge',
       'M_charge', 'other_charge', 'statute_degree'],
      dtype='object')

In [36]:
arrest_charge_combine2 = arrest_charge_combine2.reset_index()

arrest_charge_combine2.head()

,person_id,name,arrest_date,case_number,offense_date,charge_degree,charge,filing_type,filing_agency,statute,F_statute,M_statute,other_statute,F_charge_degree,M_charge_degree,other_charge_degree,F_charge,M_charge,other_charge,statute_degree
0,1,miguel hernandez,2013-08-13,[13011352CF10A],"[2013-08-13 00:00:00, 2013-08-13 00:00:00]","[(F3), (M1)]","[Aggravated Assault w/Firearm, Battery]","[SAO Information, SAO Information]","[Broward Sheriff Office, Broward Sheriff Office]","[784.021, 784.03]",[784.021],[784.03],[],[(F3)],[(M1)],[],[Aggravated Assault w/Firearm],[Battery],[],"[784(F3), 784(M1)]"
1,1,miguel hernandez,2014-07-07,[13011352CF10A],"[2013-08-13 00:00:00, 2013-08-13 00:00:00]","[(F3), (M1)]","[Aggravated Assault w/Firearm, Battery]","[SAO Information, SAO Information]","[Broward Sheriff Office, Broward Sheriff Office]","[784.021, 784.03]",[784.021],[784.03],[],[(F3)],[(M1)],[],[Aggravated Assault w/Firearm],[Battery],[],"[784(F3), 784(M1)]"
2,3,kevon dixon,2013-01-26,[13001275CF10A],[2013-01-26 00:00:00],[(F3)],[Felony Battery w/Prior Convict],[SAO Information],[Broward Sheriff Office/Pompano Beach],[784.03(2)],[784.03(2)],[],[],[(F3)],[],[],[Felony Battery w/Prior Convict],[],[],[784(F3)]
3,3,kevon dixon,2013-07-09,[13009779CF10A],"[2013-07-05 00:00:00, 2013-07-05 00:00:00, 201...","[(F2), (F3), (F3)]","[Sexual Battery / Vict 12 Yrs +, Felony Batter...","[SAO Information, SAO Information, SAO Informa...","[Broward Sheriff Office/Pompano Beach, Broward...","[794.011(5), 784.041(2)(a), 787.02(1)(a)]","[794.011(5), 784.041(2)(a), 787.02(1)(a)]",[],[],"[(F2), (F3), (F3)]",[],[],"[Sexual Battery / Vict 12 Yrs +, Felony Batter...",[],[],"[794(F2), 784(F3), 787(F3)]"
4,3,kevon dixon,2013-07-10,[13001275CF10A],[2013-01-26 00:00:00],[(F3)],[Felony Battery w/Prior Convict],[SAO Information],[Broward Sheriff Office/Pompano Beach],[784.03(2)],[784.03(2)],[],[],[(F3)],[],[],[Felony Battery w/Prior Convict],[],[],[784(F3)]


## Combine the compas, arrest and charge table

In [37]:
compass_r1ac = compass_r1.merge(arrest_charge_combine2, 
                                 left_on = 'person_id',
                                 right_on = 'person_id'
                               )

compass_r1ac['diff_compass_arrest'] = (compass_r1ac.screening_date - compass_r1ac.arrest_date).dt.days


In [38]:
(compass_r1ac.diff_compass_arrest >= 0).mean()

(compass_r1ac.diff_compass_arrest < 30).mean()

0.3726742621333094

In [39]:
compass_r1ac.shape

compass_r1ac.person_id.nunique()

10963

## Filter for compass screening within 30 days  

In [40]:
id_0 = compass_r1ac.diff_compass_arrest > 0

id_30 = compass_r1ac.diff_compass_arrest < 30

compass_r1ac_30 = compass_r1ac.loc[id_0 & id_30,:]

compass_r1ac_30.shape


(8427, 41)

In [41]:
compass_r1ac_30.case_number


0                       [13011352CF10A]
2                       [13001275CF10A]
7        [13002653MM10A, 13005330CF10A]
13       [13000570CF10A, 13006228MM10A]
36       [14002304CF10A, 14002446CF10A]
                      ...              
55725                   [13003941MM10A]
55728                   [13013317CF10A]
55729                   [13009569MM10A]
55730                   [14014708MM10A]
55734                   [14000455CF10A]
Name: case_number, Length: 8427, dtype: object

## keep only the latest arrest 

### Case of multiple arrest within 30 days

In [42]:
compass_r1ac_30_1 = \
compass_r1ac_30.groupby(['name', 'screening_date']
            ).apply(lambda x: x.drop(columns = ['name', 'screening_date']
                            ).sort_values('diff_compass_arrest').iloc[0,:]
            )

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2383231281.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(lambda x: x.drop(columns = ['name', 'screening_date']


In [43]:
compass_r1ac_30_1 = compass_r1ac_30_1.reset_index()

current_crime = compass_r1ac_30_1.copy()

In [44]:
compass_r1ac_30_1.shape

(8228, 41)

# Collect past crime together

## Create the seed matrix

In [45]:
compass_charge_seed = compass_r1ac_30_1.loc[:, ['person_id', 'screening_date', 'case_number']]

In [46]:
compass_charge_all = compass_charge_seed.merge(charge, on = 'person_id'
                                ).assign( compass_charge_diff = lambda x: (x.screening_date - x.offense_date).dt.days
                                )

## Drop the current crime

In [47]:
id1 = compass_charge_all.apply(lambda x: x.case_number_y in x.case_number_x, axis = 1)

## Collect the past crime

In [48]:
past_crime = compass_charge_all[~id1].query('compass_charge_diff > 0')

In [49]:
past_crime.shape

(68760, 28)

In [50]:
past_crime.columns

Index(['person_id', 'screening_date', 'case_number_x', 'id', 'case_type',
       'case_number_y', 'filing_date', 'offense_date', 'charge_number',
       'charge_degree', 'charge', 'date_charge_filed', 'filing_type',
       'filing_agency', 'name', 'days_since_compas', 'statute', 'F_statute',
       'M_statute', 'other_statute', 'F_charge_degree', 'M_charge_degree',
       'other_charge_degree', 'F_charge', 'M_charge', 'other_charge',
       'statute_degree', 'compass_charge_diff'],
      dtype='object')

## Create past crime variables
* Total count of past crime of total/F/M/others
* Total count of past others
* Age commiting first F/M
* average charges per year
* Juvenile count of total/F/M
* type of crimes in past
* correlations between past and current crime

In [51]:
past_crime_final = \
past_crime.drop(columns = ['case_number_x']
#         ).head(1000
        ).groupby(['person_id', 'screening_date']
        ).apply(
    
    func = lambda x: pd.Series(
  {      
      'total_offense_date_past': x.offense_date.unique().shape[0],
      'total_past':  x.offense_date.shape[0],
      'total_F_past':    x.charge_degree.str.startswith('(F').sum(),
      'total_M_past':     x.charge_degree.str.startswith('(M').sum(),
      'total_other_past': x.shape[0] - x.charge_degree.str.match(r'\((F|M)').sum(),
      
      'F_degree_past':    x[x.charge_degree.str.startswith('(F')].charge_degree.to_list(),
      'M_degree_past':    x[x.charge_degree.str.match(r'^\(M\d')].charge_degree.to_list(),
      
      'F_statute_past':   x[x.charge_degree.str.startswith('(F')].statute.to_list(),
      'M_statute_past':   x[x.charge_degree.str.startswith('(M')].statute.to_list(),
      
      'F_charge_past':   x[x.charge_degree.str.startswith('(F')].charge.to_list(),
      'M_charge_past':   x[x.charge_degree.str.startswith('(M')].charge.to_list(),
      'Statute_degree_past': x.statute_degree.to_list()
      
  }
    )
)

past_crime_final.head()

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/3173324427.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(


,,total_offense_date_past,total_past,total_F_past,total_M_past,total_other_past,F_degree_past,M_degree_past,F_statute_past,M_statute_past,F_charge_past,M_charge_past,Statute_degree_past
person_id,screening_date,,,,,,,,,,,,
1,2013-08-14,3,4,0,0,4,[],[],[],[],[],[],"[316(0), 320(0), 316(0), 316(0)]"
4,2013-04-14,7,15,3,6,6,"[(F3), (F3), (F3)]","[(M2), (M1), (M1), (M2), (M2), (M1)]","[812.014(1)(a), 893.03(2)(a)4, 951.22(1)]","[322.34(2), 843.02, 843.02, 316.061, 316.061, ...","[Grand Theft (Motor Vehicle), Possession of Co...","[Driving License Suspended, Resist/Obstruct W/...","[322(M2), 316(0), 812(F3), 843(M1), 843(M1), 3..."
5,2013-01-13,2,2,0,1,1,[],[(M2)],[],[322.03(1)],[],[Operating W/O Valid License],"[322(M2), 316(0)]"
8,2014-02-19,13,23,13,6,4,"[(F3), (F3), (F2), (F3), (F3), (F3), (F2), (F3...","[(M1), (M2), (M1), (M1), (M1), (M2)]","[812.014(2)(c ), 322.34(5), 812.13(2)(c ), 810...","[810.09, 856.021, 790.01(1), 784.03, 893.13(6)...","[Grand Theft in the 3rd Degree, Driving While ...","[Trespass Other Struct/Conve, Prowling/Loiteri...","[812(F3), 322(F3), 810(M1), 856(M2), 812(F2), ..."
9,2013-08-30,9,14,1,3,10,[(F2)],"[(M1), (M2), (M2)]",[784.045(1)(a)2],"[784.03(1), 812.014(1)(b), 322.34(2)(A)]",[Aggrav Battery w/Deadly Weapon],"[Battery, Theft/To Appropriate, Susp Drivers L...","[316(0), 316(0), 316(0), 316(0), 316(0), 316(0..."


# Collect the future crime

In [52]:
future_crime = compass_charge_all[~id1].query('compass_charge_diff < 0')

In [53]:
future_crime.shape

(17081, 28)

In [54]:
all_class = ['(0)', '(M1)', '(F3)', '(M2)', '(F2)', '(MO3)', '(F1)', '(NI0)', '(F7)', '(CO3)', '(TCX)', '(F6)', '(CT)', '(F5)']

In [55]:
future_crime_FM = future_crime[future_crime.charge_degree.str.match('\([FM][1-6]\)')]

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/363671693.py:1: SyntaxWarning: invalid escape sequence '\('
  future_crime_FM = future_crime[future_crime.charge_degree.str.match('\([FM][1-6]\)')]


In [56]:
# Create recidivism indicator, =1 if charged with a future crime (felony or misdemeanor) occurs within 2 years after the COMPAS screening
future_crime_FM_final = \
future_crime_FM[(future_crime_FM.compass_charge_diff > -365*2)
             ].groupby('person_id').size(
             ).to_frame(
             ).assign(is_recid = 1
             ).reset_index()

future_crime_FM_final.head()

,person_id,0,is_recid
0,3,3,1
1,4,9,1
2,8,4,1
3,23,7,1
4,24,2,1


# Join past, future and current crime together 

In [57]:
current_crime.shape

future_crime_FM_final.shape

past_crime_final.shape

(6117, 12)

In [58]:
full_data = \
current_crime.merge(past_crime_final, on = 'person_id', how = 'left', suffixes = ('', '_past')
            ).merge(future_crime_FM_final, on = 'person_id', how = 'left'
            )

full_data['is_recid'] = full_data.is_recid.fillna(0)

full_data.shape

full_data.head()

full_data.columns

Index([                      'name',             'screening_date',
                        'person_id',                         'id',
                            'first',                       'last',
                 'compas_person_id',             'compas_case_id',
             'compas_assessment_id',                'agency_text',
                        'scale_set',          'assessment_reason',
                     'legal_status',             'custody_status',
                   'marital_status',      'rec_supervision_level',
       'rec_supervision_level_text',                 'score_text',
                         'scale_id',         'type_of_assessment',
                        'raw_score',               'decile_score',
                      'arrest_date',                'case_number',
                     'offense_date',              'charge_degree',
                           'charge',                'filing_type',
                    'filing_agency',                    'statu

In [59]:
full_data.is_recid.mean()

0.313441905687895

In [60]:
full_data.columns

Index([                      'name',             'screening_date',
                        'person_id',                         'id',
                            'first',                       'last',
                 'compas_person_id',             'compas_case_id',
             'compas_assessment_id',                'agency_text',
                        'scale_set',          'assessment_reason',
                     'legal_status',             'custody_status',
                   'marital_status',      'rec_supervision_level',
       'rec_supervision_level_text',                 'score_text',
                         'scale_id',         'type_of_assessment',
                        'raw_score',               'decile_score',
                      'arrest_date',                'case_number',
                     'offense_date',              'charge_degree',
                           'charge',                'filing_type',
                    'filing_agency',                    'statu

In [61]:
full_data.statute_degree.head()

full_data.Statute_degree_past.head()

0     [316(0), 316(0), 316(0), 322(0), 320(0), 316(0)]
1                                                  NaN
2    [OFLD858B(MO3), OFLD53A(MO3), 812(M2), 810(F3)...
3                    [316(0), 316(0), 316(0), 810(F3)]
4             [320(0), 316(0), 316(0), 316(0), 316(0)]
Name: Statute_degree_past, dtype: object

# Exploratory analysis 

## Statute chapter

In [62]:
id1 = full_data.F_charge_degree.str.len() > 0

In [63]:
full_data.is_recid.mean()

0.313441905687895

In [64]:
full_data.groupby(full_data.F_statute.apply(lambda x: [str(i).split('.')[0] for i in x]).str[0]
        ).agg({ 
                'charge': lambda x: x.iloc[0],
                'statute': lambda x: x.shape[0],
                'is_recid': lambda x: round(x.mean(), 2)
} 
    ).sort_values('statute', 
                  ascending = False)

,charge,statute,is_recid
F_statute,,,
893,"[Possession Of Alprazolam, Possession of Canna...",1471,0.39
812,[Felony Petit Theft],1134,0.31
784,[Aggravated Battery / Pregnant],835,0.27
810,"[Burglary Unoccupied Dwelling, Battery]",715,0.39
322,"[Driving While License Revoked, Petit Theft]",375,0.37
831,[Sale/Del Counterfeit Cont Subs],138,0.28
817,"[Use Scanning Device to Defraud, Fraudulent Us...",92,0.25
790,"[Poss F/Arm Delinq, Carrying Concealed Firearm...",91,0.46
316,"[Possess Cannabis/20 Grams Or Less, Possess Dr...",91,0.23


In [65]:
full_data.groupby(full_data.F_statute.apply(lambda x: [str(i).split('.')[0] for i in x]).str[0]
        ).is_recid.agg([np.sum, np.mean, len]
).sort_values( 'len', ascending = False
)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2581929316.py:2: FutureWarning: The provided callable <function sum at 0x104a072e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).is_recid.agg([np.sum, np.mean, len]
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2581929316.py:2: FutureWarning: The provided callable <function mean at 0x104a10400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).is_recid.agg([np.sum, np.mean, len]


,sum,mean,len
F_statute,,,
893,578.0,0.392930,1471
812,351.0,0.309524,1134
784,223.0,0.267066,835
810,281.0,0.393007,715
322,140.0,0.373333,375
831,39.0,0.282609,138
817,23.0,0.250000,92
790,42.0,0.461538,91
316,21.0,0.230769,91


In [66]:
full_data.groupby(full_data.M_statute.apply(lambda x: [str(i).split('.')[0] for i in x]).str[0]
        ).is_recid.agg([np.sum, np.mean, len]
).sort_values( 'len', ascending = False
)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/737882592.py:2: FutureWarning: The provided callable <function sum at 0x104a072e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).is_recid.agg([np.sum, np.mean, len]
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/737882592.py:2: FutureWarning: The provided callable <function mean at 0x104a10400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).is_recid.agg([np.sum, np.mean, len]


,sum,mean,len
M_statute,,,
784,605.0,0.273138,2215
893,387.0,0.414347,934
316,79.0,0.146840,538
322,193.0,0.415054,465
843,156.0,0.396947,393
812,137.0,0.415152,330
810,48.0,0.400000,120
741,34.0,0.357895,95
806,33.0,0.347368,95


In [67]:
full_data.groupby(full_data.M_statute.apply(lambda x: [str(i).split('.')[0] for i in x]).str[0]
        ).is_recid.agg([np.sum, np.mean, len]
).sort_values( 'len', ascending = False
)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/737882592.py:2: FutureWarning: The provided callable <function sum at 0x104a072e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).is_recid.agg([np.sum, np.mean, len]
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/737882592.py:2: FutureWarning: The provided callable <function mean at 0x104a10400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).is_recid.agg([np.sum, np.mean, len]


,sum,mean,len
M_statute,,,
784,605.0,0.273138,2215
893,387.0,0.414347,934
316,79.0,0.146840,538
322,193.0,0.415054,465
843,156.0,0.396947,393
812,137.0,0.415152,330
810,48.0,0.400000,120
741,34.0,0.357895,95
806,33.0,0.347368,95


In [68]:
full_data.groupby(full_data.F_statute_past.apply(lambda x: [str(i).split('.')[0] for i in (x if type(x) is list else [x])]).str[0]
        ).is_recid.agg([np.sum, np.mean, len]
).sort_values( 'len', ascending = False
)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2936944341.py:2: FutureWarning: The provided callable <function sum at 0x104a072e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).is_recid.agg([np.sum, np.mean, len]
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2936944341.py:2: FutureWarning: The provided callable <function mean at 0x104a10400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).is_recid.agg([np.sum, np.mean, len]


,sum,mean,len
F_statute_past,,,
nan,361.0,0.171009,2111
893,445.0,0.450861,987
812,249.0,0.443850,561
810,195.0,0.482673,404
784,131.0,0.348404,376
322,89.0,0.400901,222
790,39.0,0.487500,80
316,14.0,0.259259,54
831,22.0,0.458333,48


In [69]:
full_data.groupby(full_data.M_statute_past.apply(lambda x: [str(i).split('.')[0] for i in (x if type(x) is list else [x])]).str[0]
        ).is_recid.agg([np.sum, np.mean, len]
).sort_values( 'len', ascending = False
)

/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2758694234.py:2: FutureWarning: The provided callable <function sum at 0x104a072e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).is_recid.agg([np.sum, np.mean, len]
/var/folders/p6/d6x2_ng95293bwjb_dlw5ptc0000gn/T/ipykernel_10275/2758694234.py:2: FutureWarning: The provided callable <function mean at 0x104a10400> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).is_recid.agg([np.sum, np.mean, len]


,sum,mean,len
M_statute_past,,,
nan,361.0,0.171009,2111
322,541.0,0.412034,1313
893,339.0,0.437419,775
784,142.0,0.344660,412
812,182.0,0.484043,376
...,...,...,...
ODVD166,0.0,0.000000,1
ODRD50105,1.0,1.000000,1
ODRD466,0.0,0.000000,1


# Add chapter as a variable

## Map for the most common statute

In [70]:
s1 = \
"""
784	Assault/Battery
812	Theft/Robery
810	Burglary and threspass
322	Driver license
831	Forgery and counterfeiting
316	Motor vehicles
893	Drug abuse
817	Fradulent
843	Obstructing justice
"""

chapter_crime_map = {i.split('\t')[0] : i.split('\t')[1] for i in s1.split('\n') if len(i) > 0}

In [71]:
chapter_crime_map

{'784': 'Assault/Battery',
 '812': 'Theft/Robery',
 '810': 'Burglary and threspass',
 '322': 'Driver license',
 '831': 'Forgery and counterfeiting',
 '316': 'Motor vehicles',
 '893': 'Drug abuse',
 '817': 'Fradulent',
 '843': 'Obstructing justice'}

## Add the current variable

In [72]:
def convert_chapter(var, suffix = '', crime_map = None):

    suffix_f = '_' + suffix if len(suffix) > 0 else ''
    
    crime_chapter = full_data[var].apply(lambda x: [str(i).split('.')[0] 
                                         for i in (x if type(x) is list else [x])]
                                        ).astype(str)
    
    v1 = CountVectorizer()

    raw_chapter_0 = v1.fit_transform(crime_chapter).toarray()
    
    raw_chapter_1 = pd.DataFrame(raw_chapter_0, columns = pd.Series(v1.get_feature_names_out()))
    
    if crime_map is not None:
        
        major_crime = raw_chapter_1.loc[:, chapter_crime_map.keys()].rename(columns = crime_map)

        other_crime = \
        raw_chapter_1.drop(columns = crime_map.keys()
                            ).sum(axis = 1
                            )

        return pd.concat([major_crime, other_crime], axis = 1
                ).rename(columns = {0: 'Other'}
                ).rename(columns = lambda x: x + suffix_f)
        
    else:
    
        return raw_chapter_1.rename(columns = lambda x: x + suffix_f)

In [73]:
def convert_statute(var, suffix = '', crime_map = None):

    suffix_f = '_' + suffix if len(suffix) > 0 else ''
    
    crime_chapter = full_data[var].apply(lambda x: [str(i).replace('(', '_').replace(')', '') for i in (x if type(x) is list else [x])]).astype(str)
    
    v1 = CountVectorizer()

    raw_chapter_0 = v1.fit_transform(crime_chapter).toarray()
    
    raw_chapter_1 = pd.DataFrame(raw_chapter_0, columns = pd.Series(v1.get_feature_names_out()))
    
    if crime_map is not None:
        
        major_crime = raw_chapter_1.loc[:, chapter_crime_map.keys()].rename(columns = crime_map)

        other_crime = \
        raw_chapter_1.drop(columns = crime_map.keys()
                            ).sum(axis = 1
                            )

        return pd.concat([major_crime, other_crime], axis = 1
                ).rename(columns = {0: 'Other'}
                ).rename(columns = lambda x: x + suffix_f)
        
    else:
    
        return raw_chapter_1.rename(columns = lambda x: x + suffix_f)

In [74]:
crime_chapter2 = full_data['statute_degree'].apply(lambda x: [str(i).replace('(', '_').replace(')', '') for i in (x if type(x) is list else [x])]).astype(str)
    
v2 = CountVectorizer()

raw_chapter_01 = v2.fit_transform(crime_chapter2).toarray()

raw_chapter_01.shape

raw_chapter_11 = pd.DataFrame(raw_chapter_01, columns = pd.Series(v2.get_feature_names_out()))

In [75]:
raw_chapter_11.head()

,122_mo3,316_0,316_f1,316_f2,316_f3,316_m1,316_m2,316_tc4,319_f3,320_0,...,ohwd13327_mo3,ohwd13327a_mo3,ohwd13327b_mo3,ohwd13327c1_mo3,ohwd13331_mo3,ohwd9702a_mo3,olhd1439_mo3,oopd331a_mo3,opbd13311_mo3,opbd13311a_mo3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
current_statute_degree = convert_statute('statute_degree', 'current')

current_statute_degree.shape

current_statute_degree.head()

past_statute_degree = convert_statute('Statute_degree_past', 'past')

past_statute_degree.shape

past_statute_degree.head()

,1003277a_m2_past,1003_m2_past,113_m2_past,122_mo3_past,12569_co3_past,151_mo3_past,1655c_mo3_past,1_mo3_past,205_co3_past,21106_co3_past,...,opbd9018b_mo3_past,opbd9023_mo3_past,opbd9610a1_mo3_past,opbd9760_past,opbd9760a2_mo3_past,opbd9813a_mo3_past,opkd17_past,opld312_mo3_past,osnd7268_mo3_past,txxu_tcx_past
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
M_chapter_list_current = convert_chapter('M_statute', 'current_M',chapter_crime_map)

F_chapter_list_current = convert_chapter('F_statute', 'current_F',chapter_crime_map)

M_chapter_list_past = convert_chapter('M_statute_past', 'past_M',chapter_crime_map)

F_chapter_list_past = convert_chapter('F_statute_past', 'past_F',chapter_crime_map)

In [78]:
crime_chapter_all = pd.concat([M_chapter_list_current, 
                               F_chapter_list_current,
                               M_chapter_list_past,
                               F_chapter_list_past,
                               current_statute_degree,
                               past_statute_degree], axis = 1)

In [79]:
current_statute_degree.shape[1] + past_statute_degree.shape[1]

732

In [80]:
crime_chapter_all.shape

(8228, 772)

## Convert a dummy variable to count occurence only

In [81]:
crime_chapter_all_d = (crime_chapter_all > 0).astype(int).rename(columns = lambda x: x + '_d')

In [82]:
crime_chapter_all_d['Drug abuse_current_F_d'].value_counts()

Drug abuse_current_F_d
0    6660
1    1568
Name: count, dtype: int64

# Combine data together

In [83]:
len([i for i in crime_chapter_all.columns.to_list() if i.endswith('_F')])

20

In [84]:
full_data_chapter = pd.concat([full_data, crime_chapter_all], axis = 1)


# Add people related variables

In [85]:
people['compas_screening_date'] = people.compas_screening_date.str.split(' ').str[0]
full_data_chapter['screening_date'] = full_data_chapter.screening_date.astype('string').str.split(' ').str[0]

In [86]:
people_1 = \
people.loc[:, ['first', 'last', 'sex', 'race', 'dob', 'age', 'juv_fel_count', 
              'juv_misd_count', 'juv_other_count', 'compas_screening_date',
              'decile_score'
              ]
          ].drop_duplicates(
          )

In [87]:
merge_test = \
full_data_chapter.merge(people_1, 
                        left_on = ['first', 'last', 'screening_date', 'decile_score'],
                        right_on = ['first', 'last', 'compas_screening_date', 'decile_score']
                        ,how = 'left'                        
                       )

merge_test.shape

(8229, 835)

In [88]:
merge_test.groupby(['first', 'last', 'screening_date', 'decile_score']
                  ).size(
                  ).sort_values( ascending = False
                  )
                  
id1 = (merge_test['first'] == 'donald') & (merge_test['last'] == 'shupert')

id1.sum()

2

In [89]:
# Compute age based on dob and screening_date
merge_test['age'] = ((pd.to_datetime(merge_test.screening_date) - pd.to_datetime(merge_test.dob)).dt.days / 365).astype(int)

# Create dummies for gender and marital status
merge_test['sex_female'] = (merge_test.sex == 'Female').astype(int)
merge_test = pd.concat([merge_test, 
                       pd.get_dummies(merge_test.marital_status, prefix = 'mariage', dtype=int)
                       ], 
                       axis = 1)


In [90]:
final = merge_test[~id1]
final.shape
final.to_csv('../data/data_clean.csv', index = False)

In [91]:
# Table OA.3: Summary Statistics
charges_current = final.iloc[:, 55:65] + final.iloc[:, 65:75].values
charges_current.columns = charges_current.columns.str.replace(r'_current.*', '', regex=True)
current_charges_total = charges_current.sum()

# Categorical counts
gender_counts = final.sex.value_counts()
recid_counts = final.is_recid.value_counts().sort_index()
race_counts = final.race.value_counts()
marital_counts = final.marital_status.value_counts()

# Charge degree categories
all_charge_degrees = []
for item in final.charge_degree:
    if isinstance(item, list):
        all_charge_degrees.extend(item)
    elif pd.notna(item):
        all_charge_degrees.append(item)

def categorize_charge_degree(degree):
    if pd.isna(degree):
        return 'Other'
    degree = str(degree).upper()
    if '(F1)' in degree:
        return 'Felony 1st'
    elif '(F2)' in degree:
        return 'Felony 2nd'
    elif any(f'(F{i})' in degree for i in [3, 4, 5, 6, 7]):
        return 'Felony 3rd+'
    elif '(M1)' in degree:
        return 'Misdemeanor 1st'
    elif '(M2)' in degree:
        return 'Misdemeanor 2nd'
    elif any(x in degree for x in ['(CO3)', '(MO3)', '(TC4)', '(TCX)']):
        return 'Ordinance/Traffic'
    elif any(x in degree for x in ['(NI0)', '(0)']):
        return 'Other'
    else:
        return 'Other'

charge_degree_categories = [categorize_charge_degree(d) for d in all_charge_degrees]
charge_degree_counts = pd.Series(charge_degree_categories).value_counts()

# Continuous statistics
age_stats = final.age.describe()
score_stats = final.raw_score.describe()
total_past_stats = final.total_past.describe()

# Build summary table
charge_labels = ['Assault/Battery', 'Theft/Robbery', 'Burglary', 'Drug abuse', 'Driver license', 'Motor vehicles', 'Forgery']
charge_map = {'Assault/Battery': 'Assault/Battery', 'Theft/Robery': 'Theft/Robbery', 
              'Burglary and threspass': 'Burglary', 'Drug abuse': 'Drug abuse',
              'Driver license': 'Driver license', 'Motor vehicles': 'Motor vehicles',
              'Forgery and counterfeiting': 'Forgery', 'Fradulent': 'Fraudulent',
              'Obstructing justice': 'Obstruct justice', 'Other': 'Other'}

summary_data = [
    ['Current Charges'] + charge_labels,
    [''] + [int(current_charges_total.get(k, 0)) for k, v in charge_map.items() if v in charge_labels],
    ['', 'Fraudulent', 'Obstruct justice', 'Other', 'Forgery', '', '', ''],
    ['', int(current_charges_total.get('Fradulent', 0)), int(current_charges_total.get('Obstructing justice', 0)), 
    int(current_charges_total.get('Other', 0)), int(current_charges_total.get('Forgery and counterfeiting', 0)), '', '', ''],
    ['Charge Degrees', 'Felony 1st', 'Felony 2nd', 'Felony 3rd+', 'Misd. 1st', 'Misd. 2nd', 'Ord./Traffic', 'Other'],
    ['', charge_degree_counts.get('Felony 1st', 0), charge_degree_counts.get('Felony 2nd', 0),
     charge_degree_counts.get('Felony 3rd+', 0), charge_degree_counts.get('Misdemeanor 1st', 0),
     charge_degree_counts.get('Misdemeanor 2nd', 0), charge_degree_counts.get('Ordinance/Traffic', 0),
     charge_degree_counts.get('Other', 0)],
    ['Gender', 'Male', 'Female', '', '', '', '', ''],
    ['', gender_counts.get('Male', 0), gender_counts.get('Female', 0), '', '', '', '', ''],
    ['is_recid', '0', '1', '', '', '', '', ''],
    ['', recid_counts.get(0.0, 0), recid_counts.get(1.0, 0), '', '', '', '', ''],
    ['Race', 'African American', 'Asian', 'Caucasian', 'Hispanic', 'Native American', 'Other', ''],
    ['', race_counts.get('African-American', 0), race_counts.get('Asian', 0), race_counts.get('Caucasian', 0),
     race_counts.get('Hispanic', 0), race_counts.get('Native American', 0), race_counts.get('Other', 0), ''],
    ['Marital Status', 'Married', 'Single', 'Divorced', 'Separated', 'Widowed', 'Unknown', ''],
    ['', marital_counts.get('Married', 0), marital_counts.get('Single', 0), marital_counts.get('Divorced', 0),
     marital_counts.get('Separated', 0), marital_counts.get('Widowed', 0), marital_counts.get('Unknown', 0), ''],
    ['', 'Min.', '25% Quantile', 'Median', 'Mean', '75% Quantile', 'Max', ''],
    ['Raw Score'] + [int(score_stats[k]) for k in ['min', '25%', '50%', 'mean', '75%', 'max']] + [''],
    ['Prior Charge Count'] + [int(total_past_stats[k]) for k in ['min', '25%', '50%', 'mean', '75%', 'max']] + [''],
    ['Age'] + [int(age_stats[k]) for k in ['min', '25%', '50%', 'mean', '75%', 'max']] + ['']
]

# Convert to DataFrame and generate LaTeX
summary_df = pd.DataFrame(summary_data)
latex_table = summary_df.to_latex(
    index=False, header=False,
    caption="Summary Statistics: Cleaned Dataset",
    label="tab:summary_stats", position="htbp", column_format='l' + 'c' * 7
)
with open('../tab/tab_oa3.tex', 'w') as f:
    f.write(latex_table)
    
print(latex_table)


\begin{table}[htbp]
\caption{Summary Statistics: Cleaned Dataset}
\label{tab:summary_stats}
\begin{tabular}{lccccccc}
\toprule
\midrule
Current Charges & Assault/Battery & Theft/Robbery & Burglary & Drug abuse & Driver license & Motor vehicles & Forgery \\
 & 3824 & 2583 & 1290 & 3677 & 1593 & 1476 & 275 \\
 & Fraudulent & Obstruct justice & Other & Forgery &  &  &  \\
 & 312 & 882 & 2820 & 275 &  &  &  \\
Charge Degrees & Felony 1st & Felony 2nd & Felony 3rd+ & Misd. 1st & Misd. 2nd & Ord./Traffic & Other \\
 & 380 & 1951 & 7196 & 6837 & 2377 & 157 & 2633 \\
Gender & Male & Female &  &  &  &  &  \\
 & 6508 & 1719 &  &  &  &  &  \\
is_recid & 0 & 1 &  &  &  &  &  \\
 & 5648 & 2579 &  &  &  &  &  \\
Race & African American & Asian & Caucasian & Hispanic & Native American & Other &  \\
 & 4109 & 45 & 2797 & 720 & 21 & 535 &  \\
Marital Status & Married & Single & Divorced & Separated & Widowed & Unknown &  \\
 & 991 & 6380 & 328 & 194 & 36 & 34 &  \\
 & Min. & 25% Quantile & Median & Mea